<a href="https://colab.research.google.com/github/S-HATANO1970/otemae/blob/main/%E6%83%85%E5%A0%B1%E6%B4%BB%E7%94%A82_%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ニューラルネットワーク
機械学習モデルとの比較

In [2]:
import numpy as np

class SimpleNeuralNetwork:
    """素朴なニューラルネットワーク実装（3層、ReLU活性化関数）"""
    def __init__(self, input_size:int, hidden_size:int, output_size:int, learning_rate:float=0.01, fixed_initial_weights:bool=False):
        """
        コンストラクタ
        :param input_size: 入力層のニューロン数
        :param hidden_size: 隠れ層のニューロン数
        :param output_size: 出力層のニューロン数
        :param learning_rate: 学習率
        :param fixed_initial_weights: 初期重みを固定するかどうか
        """
        # 重みの初期化
        if(fixed_initial_weights):
            np.random.seed(42)
        self.W1:np.ndarray = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
        if(fixed_initial_weights):
            np.random.seed(42)
        self.W2:np.ndarray = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)
        # バイアスの初期化
        self.b1:np.ndarray = np.zeros((1, hidden_size))
        self.b2:np.ndarray = np.zeros((1, output_size))

        # 学習率
        self.learning_rate: float = learning_rate
        # 中間層の出力と活性化関数の値
        self.z1: np.ndarray
        self.a1: np.ndarray
        # self.z2: np.ndarray
        self.a2: np.ndarray
        # 損失の履歴
        # 学習過程での損失を記録するリスト
        self.losses: list = []

    def relu(self, x: np.ndarray) -> np.ndarray:
        """
        ReLU活性化関数
        負の値を0にし、正の値はそのまま返す
        :param x: 入力データ
        :return: ReLU適用後の出力
        """
        return np.maximum(0, x)

    def relu_derivative(self, x: np.ndarray) -> np.ndarray:
        """
        ReLUの微分
        :param x: 入力データ
        :return: ReLUの微分値
        """
        return (x > 0).astype(float)

    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        順伝播
        :param X: 入力データ (サンプル数, 入力次元)
        :return: 出力データ (サンプル数, 出力次元)
        """
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        self.a2 = np.dot(self.a1, self.W2) + self.b2
        # self.a2 = self.z2  # 出力層は線形
        return self.a2

    def backward(self, X: np.ndarray, y: np.ndarray, output: np.ndarray) -> None:
        """
        逆伝播
        :param X: 入力データ (サンプル数, 入力次元)
        :param y: 正解データ (サンプル数, 出力次元)
        :param output: ネットワークの出力 (サンプル数, 出力次元)
        """
        m = X.shape[0]

        dz2 = output - y
        dW2 = (1/m) * np.dot(self.a1.T, dz2)
        db2 = (1/m) * np.sum(dz2, axis=0, keepdims=True)

        da1 = np.dot(dz2, self.W2.T)
        dz1 = da1 * self.relu_derivative(self.z1)
        dW1 = (1/m) * np.dot(X.T, dz1)
        db1 = (1/m) * np.sum(dz1, axis=0, keepdims=True)

        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1

    def fit(self, X: np.ndarray, y: np.ndarray, epochs: int=1000, verbose: bool=False, print_weights_every: int=10) -> None:
        """
        学習
        :param X: 入力データ (サンプル数, 入力次元)
        :param y: 正解データ (サンプル数, 出力次元)
        :param epochs: 学習エポック数
        :param verbose: 学習過程を表示するかどうか
        :param print_weights_every: 学習過程を表示する頻度
        """
        for epoch in range(epochs):
            output = self.forward(X)
            loss = np.mean((output - y)**2)
            self.losses.append(loss)
            self.accuracy = np.mean(np.round(output) == y)

            if epochs > 0:
                self.backward(X, y, output)

            if verbose:
                if 0 < epoch < 10:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}, Output:{output.reshape(1,y.size).round(2)}")
                if epoch <= 100 and epoch % print_weights_every == 0:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}, Output:{output.reshape(1,y.size).round(2)}")
                elif epoch <= 1000 and epoch % (print_weights_every*10) == 0:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}, Output:{output.reshape(1,y.size).round(2)}")
                elif epoch % (print_weights_every*50) == 0:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}, Output:{output.reshape(1,y.size).round(2)}")
                elif epoch == epochs - 1:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}, Output:{output.reshape(1,y.size).round(2)}")

    def predict(self, X: np.ndarray) -> np.ndarray:
        """予測"""
        return self.forward(X)

def print_y_vs_y_pred(y: np.ndarray, y_pred: np.ndarray) -> None:
    """
    正解データと予測結果を表示
    :param y: 正解データ
    :param y_pred: 予測結果
    """
    print("正解: 予測: 判定")
    justification = ""
    y_pred_int = 0
    correct_count = 0
    for i in range(y_pred.size):
        y_pred_int = int(y_pred[i].round(0))
        # 予測結果を四捨五入して正解と比較
        if y_pred_int == y[i]:
            justification = "○"
            correct_count += 1
        else:
            justification = "×"
        print(f"{y[i]}:{y_pred_int}:{justification}")
    print(f"正解率: {correct_count / y.size:.2f} ")

### サンプルデータと学習の実行
'''
X:学習時間・睡眠時間とy:不合格・合格についてのデータ
x:学習時間・睡眠時間
y:不合格 0, 合格 1
'''
X = np.array([
    [1,10],
    [1.5,8],
    [2,6],
    [3,5],
    [3.5,7],
    [4.5,6],
    [5.5,7],
    [5.8,8],
    [6.5,7],
    [7,5],
    [7.5,4],
    [8,7],
    [8.5,6]
])

y = np.array([
    [0],
    [0],
    [1],
    [0],
    [1],
    [1],
    [1],
    [1],
    [0],
    [1],
    [1],
    [0], # 1にしてみる
    [0]  # 1にしてみる
])
'''
# --- 1. シンプルなサンプルデータの準備 ---
# 入力データ X: (サンプル数, input_size)
# 例: ORゲートのような論理ゲートを学習
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

# 正解データ y: (サンプル数, output_size)
# 例: 論理和 (OR) の結果
y = np.array([
    [0],
    [1],
    [1],
    [1]
])
'''
# --- 2. ネットワークのパラメータ設定 ---
input_size = X.shape[1]      # 入力データの次元 (2)
hidden_size = 3              # 隠れ層のニューロン数 (任意に設定)
output_size = y.shape[1]     # 出力データの次元 (1)
learning_rate = 0.005        # 学習率
epochs = 20000               # エポック数

# --- 3. モデルの初期化と学習前の重み・バイアスの確認 ---
print("--- モデル初期化直後の重みとバイアス ---")
model = SimpleNeuralNetwork(input_size, hidden_size, output_size, learning_rate, fixed_initial_weights=True)
print(f"W1 (初期):\n{model.W1.round(4)}")
print(f"b1 (初期):\n{model.b1.round(4)}")
print(f"W2 (初期):\n{model.W2.round(4)}")
print(f"b2 (初期):\n{model.b2.round(4)}")

# --- 4. モデルの学習（途中の重み・バイアスも表示） ---
print("\n--- 学習開始 ---")
# verbose=Trueで損失の推移を表示
model.fit(X, y, epochs=epochs, verbose=True)
print("--- 学習終了 ---")

# --- 5. 学習後の重みとバイアスの最終確認 ---
print("\n--- 学習終了後の重みとバイアス ---")
print(f"W1 (学習後):\n{model.W1.round(4)}")
print(f"b1 (学習後):\n{model.b1.round(4)}")
print(f"W2 (学習後):\n{model.W2.round(4)}")
print(f"b2 (学習後):\n{model.b2.round(4)}")

# --- 6. 学習後の予測結果の確認 ---
print("\n--- 入力 ---")
print(f"{X}")
print("\n--- 学習後の予測結果 ---")
y_pred = np.array(model.predict(X))
print(y_pred.round(2))
print("\n--- 正解と予測結果の比較 ---")
print_y_vs_y_pred(y.reshape(y.size), y_pred.reshape(y.size))

# --- 7. 機械学習との比較 ---
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
y = y.reshape(y.size)
model.fit(X,y)
print("\n--- 回帰式の重みとバイアス ---")
print(f"W:\n{model.coef_.round(4)}")
print(f"b:\n{model.intercept_.round(4)}")
y_pred = np.array(model.predict(X)).reshape(y.size)
print("\n--- 正解と予測結果の比較 ---")
print_y_vs_y_pred(y, y_pred)


--- モデル初期化直後の重みとバイアス ---
W1 (初期):
[[ 0.4967 -0.1383  0.6477]
 [ 1.523  -0.2342 -0.2341]]
b1 (初期):
[[0. 0. 0.]]
W2 (初期):
[[ 0.4056]
 [-0.1129]
 [ 0.5288]]
b2 (初期):
[[0.]]

--- 学習開始 ---
Epoch 0, Loss: 32.3815, Output:[[6.38 5.24 4.11 4.1  5.36 5.41 6.45 7.11 6.99 6.28 6.06 7.81 7.59]]
Epoch 1, Loss: 0.9071, Output:[[0.7  0.57 0.44 0.47 0.54 0.84 1.06 1.07 1.35 1.64 1.86 1.79 2.01]]
Epoch 2, Loss: 0.5444, Output:[[0.17 0.14 0.1  0.14 0.13 0.41 0.55 0.51 0.82 1.21 1.47 1.23 1.49]]
Epoch 3, Loss: 0.5166, Output:[[0.06 0.04 0.03 0.06 0.04 0.31 0.43 0.37 0.69 1.1  1.37 1.09 1.35]]
Epoch 4, Loss: 0.5082, Output:[[0.04 0.03 0.02 0.04 0.03 0.28 0.4  0.34 0.66 1.07 1.34 1.05 1.31]]
Epoch 5, Loss: 0.5014, Output:[[0.05 0.03 0.02 0.03 0.03 0.28 0.39 0.33 0.65 1.05 1.32 1.03 1.3 ]]
Epoch 6, Loss: 0.4950, Output:[[0.06 0.04 0.03 0.03 0.04 0.27 0.39 0.33 0.64 1.04 1.31 1.02 1.29]]
Epoch 7, Loss: 0.4889, Output:[[0.07 0.05 0.04 0.03 0.05 0.27 0.39 0.33 0.64 1.04 1.3  1.01 1.28]]
Epoch 8, Loss: 0.4831, 